# 清洗已分段数据

In [4]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('Warning')

In [5]:
sample_data_dir = 'F:/Database/Multimodal_data/mne_data/MNE-sample-data/' # 设置已有数据路径
sample_data_dir = pathlib.Path(sample_data_dir)
epochs = mne.read_epochs(pathlib.Path(sample_data_dir / 'out_data') / 'epochs_epo.fif')
epochs

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [6]:
epochs.apply_baseline((None, 0)) 

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


In [7]:
epochs.plot()

<MNEBrowseFigure size 2560x1494 with 4 Axes>

## 基于通道信号幅度剔除伪迹

In [8]:
reject_criteria = dict(mag=3000e-15,     # 3000 fT
                       grad=3000e-13,    # 3000 fT/cm
                       eeg=150e-6,       # 150 µV
                       eog=200e-6)       # 200 µV 

flat_criteria = dict(mag=1e-15,          # 1 fT
                     grad=1e-13,         # 1 fT/cm
                     eeg=1e-6)           # 1 µV 

In [9]:
epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Left: 67Visual/Right: 57
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


- 从结果可以看出，事件数量从320变成了271，不符合要求的分段都被剔除了
- 由于EOG伪迹影响，相当多的分段被剔除了

In [27]:
epochs.plot_drop_log() # 查看丢弃的数据统计

<Figure size 640x480 with 1 Axes>

In [22]:
epochs['Visual'].plot_image() # 查看视觉通道的数据

[<Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>]

In [12]:
epochs.plot_sensors(ch_type='eeg') # 查看剩余的eeg通道的电极排布

<Figure size 640x640 with 1 Axes>

In [36]:
epochs['Visual'].plot_image(picks='EEG 060') # 查看EEG 060通道的数据

[<Figure size 2560x1448 with 4 Axes>]

## SSP
尝试保留大部分分段的情况下，消除EOG伪迹的影响

In [28]:
bids_root = pathlib.Path(sample_data_dir / 'out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root) # 设置BIDS数据参数

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40) # 设置滤波参数

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=1, average=True) # 计算心电图相关的proj

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1, n_eeg=1, average=True) # 计算眼电图相关的proj

In [29]:
eog_projs # 查看眼电图相关的proj结果

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203, exp. var : 92.61%>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102, exp. var : 88.38%>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59, exp. var : 98.97%>]

In [30]:
projs = eog_projs + ecg_projs # 合并proj
projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203, exp. var : 92.61%>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102, exp. var : 88.38%>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59, exp. var : 98.97%>,
 <Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203, exp. var : 78.55%>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102, exp. var : 94.80%>,
 <Projection | ECG-eeg--0.200-0.400-PCA-01, active : False, n_channels : 59, exp. var : 87.50%>]

In [31]:
epochs.add_proj(projs) # 添加proj
epochs.plot()

<MNEBrowseFigure size 2560x1494 with 5 Axes>

In [32]:
epochs_cleaned = epochs.copy().apply_proj() # 应用proj

epochs_cleaned['Visual'].plot_image() # 查看清洗后的数据 

[<Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>]

In [35]:
epochs_cleaned['Visual'].plot_image(picks='EEG 060') # 查看清洗后的EEG 060通道的数据

[<Figure size 2560x1448 with 4 Axes>]

In [34]:
epochs_cleaned.plot_drop_log() # 查看丢弃的数据统计

<Figure size 640x480 with 1 Axes>

在使用了SSP后，以'EEG 060'通道为例，其原来波峰的位置变为了波谷，说明EOG对该通道的影响得到了有效抑制。其余通道效果同理。

## ICA--主成分分析去伪迹
重新加载原始数据，并进行频率1.0Hz的高通滤波，对后续ICA的性能更有利

In [37]:
bids_root = pathlib.Path(sample_data_dir / 'out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root) # 设置BIDS数据参数

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1, h_freq=40) # 1Hz的高通滤波对ICA分析更有利

Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,mne_anonymize
Participant,sub-01
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers, 9 Stimulus, 59 EEG, 1 EOG"
Bad channels,"MEG 2443, EEG 053"
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


读取分段并提取出那些被保留的分段（在本次实验中，所有分段都被保留了，因为在之前的操作中并没有执行任何剔除分段后的保存操作。但是在真实实验情况下，只想对实际输入的分段进行ICA分析）

In [38]:
epochs = mne.read_epochs(pathlib.Path(sample_data_dir / 'out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection # 选择部分数据进行ICA分析
epochs_selection

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

保留与保留分段对应的事件子集

In [39]:
events, event_id = mne.events_from_annotations(raw) 
events = events[epochs_selection]

为ICA创建分段，所有参数应与要清洗的分段完全匹配

In [40]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True) 

执行ICA

In [41]:
epochs_ica.info

Measurement date,"December 03, 2002 19:01:10 GMT"
Experimenter,mne_anonymize
Participant,sub-01
Digitized points,146 points
Good channels,"203 Gradiometers, 102 Magnetometers, 9 Stimulus, 59 EEG, 1 EOG"
Bad channels,"MEG 2443, EEG 053"
EOG channels,EOG 061
ECG channels,Not available
Sampling frequency,600.61 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


In [49]:
n_components = 0.99 # 设置ICA分解的成分数，通常应该设置的更高，比如0.999
method = 'picard' # 设置ICA分解的方法，需要先'pip install python-picard'安装
max_iter = 500 # 设置ICA分解的最大迭代次数，通常应该设置的更高，比如500或1000
fit_params = dict(fastica_it=5) # 设置ICA分解的参数
random_state = 42 # 设置随机种子

ica = mne.preprocessing.ICA(n_components=n_components,
                            # max_pca_components=300, #没有这个参数
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state) 
ica.fit(epochs_ica)

C:\Users\pomelo\AppData\Local\Temp\ipykernel_12004\1174959424.py:13: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs_ica)
d:\Miniconda3\envs\MF\lib\site-packages\picard\solver.py:213: UserWarning: Picard did not converge. Final gradient norm : 2.805e-05. Requested tolerance : 1e-07. Consider increasing the number of iterations or the tolerance.
  warnings.warn('Picard did not converge. Final gradient norm : %.4g.'


Method,picard
Fit parameters,fastica_it=5max_iter=500
Fit,500 iterations on epochs (153920 samples)
ICA components,233
Available PCA components,364
Channel types,"mag, grad, eeg"
ICA components marked for exclusion,—


迭代次数不够，算法没有收敛，建议增加迭代次数，此处是为了学习，暂时到此为止了

In [50]:
ica.plot_components(inst=epochs) # 查看ICA分解的结果

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x731 with 13 Axes>]

心电和眼电模式检测

In [52]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None, baseline=(None, -0.2), tmin=-0.5, tmax=0.5) # 创建心电图相关的epochs
ecg_evoked = ecg_epochs.average() # 计算心电图相关的evoked
ecg_inds, ecg_scores = ica.find_bads_ecg(ecg_epochs, method='ctps') # 查找心电图相关的bad components

eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None, baseline=(None, -0.2), tmin=-0.5, tmax=0.5) # 创建眼电图相关的epochs
eog_evoked = eog_epochs.average() # 计算眼电图相关的evoked
eog_inds, eog_scores = ica.find_bads_eog(eog_epochs) # 查找眼电图相关的bad components

components_to_exclude = ecg_inds + eog_inds # 合并bad components
ica.exclude = components_to_exclude # 设置ICA的exclude参数

绘制自动伪迹检测分数

In [53]:
ica.plot_scores(ecg_scores) 

<Figure size 2560x1448 with 1 Axes>

In [54]:
ica.plot_scores(eog_scores)

<Figure size 2560x1448 with 1 Axes>

绘制ICA来源

In [62]:
ica.plot_sources(ecg_evoked) 

<Figure size 2560x1448 with 1 Axes>

绘制原始数据和清洗后数据的叠加图

In [56]:
ica.plot_overlay(ecg_evoked)

C:\Users\pomelo\AppData\Local\Temp\ipykernel_12004\1509005474.py:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.plot_overlay(ecg_evoked)


<Figure size 2560x1448 with 3 Axes>

## 练习

- 可视化伪迹检测分数、ICA源以及基于EOG分段的原始和清洗数据的叠加图

In [ ]:
ica.plot_scores(eog_scores) 

<Figure size 2560x1448 with 1 Axes>

眼电图的伪迹检测分数正负都有，而心电图的只有正的是为什么?

眼电图（EOG）伪迹检测分数正负都有
1. 信号特性：眼电图信号（EOG）通常包含多种类型的伪迹，如眨眼、眼动等。这些伪迹在不同的方向和幅度上会影响EEG信号，导致ICA分解后的成分在相关性分析中表现出正负不同的分数。例如，眨眼信号通常在前额区域产生较大的正向偏转，而眼动信号可能在不同的方向上产生正负不同的偏转。
2. ICA成分的多样性：ICA分解的目的是将混合信号分解为独立的成分。对于EOG伪迹，不同的ICA成分可能捕捉到不同方向和幅度的伪迹信号，因此在相关性分析中会表现出正负不同的分数。

心电图（ECG）伪迹检测分数只有正的
1. 信号特性：心电图信号（ECG）通常表现为周期性的QRS复合波，这些波形在EEG数据中表现为周期性的正向偏转。由于心电图信号的周期性和一致性，ICA分解后的成分在相关性分析中通常表现为正的分数。
2. 检测方法：MNE中用于检测ECG伪迹的方法（如find_bads_ecg）通常基于信号的周期性和一致性来识别伪迹成分。这些方法在计算相关性时，通常会得到正的分数，因为心电图信号的特征在EEG数据中表现为一致的正向偏转。

In [63]:
ica.plot_sources(eog_evoked) # 从这个图与心电图的ICA源图对比可以看出，与所查看信号相关的成分会有较大的波动，更加突出显示

<Figure size 640x480 with 1 Axes>

In [64]:
ica.plot_overlay(eog_evoked) # 从叠加图可以看出，清洗前的数据会在眼电图相关的时间段有较大的波动，清洗后的数据则会减小

C:\Users\pomelo\AppData\Local\Temp\ipykernel_12004\3418402708.py:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.plot_overlay(eog_evoked)


<Figure size 2560x1448 with 3 Axes>